<h1 style="font-family:Times New Roman; font-size:50px"> VITAL<img src="logo.png" alt="Drawing" style="width: 90px;" align="right"/>

<h1 style=font-size:25px>Mediapipe</h1>
    
<p style=font-size:18px>Mediapipe, one of the applications BlazePose can enable is fitness. More specifically - pose classification and repetition counting. In this section we’ll provide basic guidance on building a custom pose classifier with the help of Colabs and wrap it in a simple fitness demo within ML Kit quickstart app. Push-ups and squats are used for demonstration purposes as the most common exercises.

<p style=font-size:18px>We picked the k-nearest neighbors algorithm (k-NN) as the classifier. It’s simple and easy to start with. The algorithm determines the object’s class based on the closest samples in the training set.

<p style=font-size:16px>To build it, one needs to:

>1. Collect image samples of the target exercises and run pose prediction on them,
>2. Convert obtained pose landmarks to a representation suitable for the k-NN classifier and form a training set using these Colabs,
>3. Perform the classification itself followed by repetition counting (e.g., in the ML Kit quickstart app).

<img src="pushup_squats.gif" alt="Drawing" style="width:450px;"  align="center">


## Pose Landmark Model
<p style= "font-size:18px;">The landmark model in MediaPipe Pose predicts the location of 33 pose landmarks (see figure below).
<br/>
<center><img src="PLM.png" alt="Drawing" style="width:620px;"/>

## Resources
>[Mediapipe Documentation](https://google.github.io/mediapipe/)
<br>
>[Pose Classification](https://google.github.io/mediapipe/solutions/pose.html)

<center><span style="font-family:Product Sans; font-size:1em;">Copyright © 2021 Anirudh Roy, Arjit Mehta, Hitesh Beniwal, Palakshee, Saksham Jain</span></center>
<center><span style="font-family:Product Sans; font-size:1em;">All Rights Reserved.</span>

In [2]:
import cv2
import mediapipe as mp
import time
import math


class poseDetector():

    def __init__(self, mode=False, model_complexity=1,upBody=False, smooth_landmarks=True,smooth=True,
                 detectionCon=0.5, trackCon=0.5):

        self.mode = mode
        self.upBody = upBody
        self.smooth = smooth
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpDraw = mp.solutions.drawing_utils
        self.mpPose = mp.solutions.pose
        self.pose = self.mpPose.Pose(self.mode, self.upBody, self.smooth,
                                     self.detectionCon, self.trackCon)

    def findPose(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.pose.process(imgRGB)
        if self.results.pose_landmarks:
            if draw:
                self.mpDraw.draw_landmarks(img, self.results.pose_landmarks,
                                           self.mpPose.POSE_CONNECTIONS)
        return img

    def findPosition(self, img, draw=True):
        self.lmList = []
        if self.results.pose_landmarks:
            for id, lm in enumerate(self.results.pose_landmarks.landmark):
                h, w, c = img.shape
                # print(id, lm)
                cx, cy = int(lm.x * w), int(lm.y * h)
                self.lmList.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 5, (255, 0, 0), cv2.FILLED)
        return self.lmList

    def findAngle(self, img, p1, p2, p3, draw=True):

        # Get the landmarks
        x1, y1 = self.lmList[p1][1:]
        x2, y2 = self.lmList[p2][1:]
        x3, y3 = self.lmList[p3][1:]

        # Calculate the Angle
        angle = math.degrees(math.atan2(y3 - y2, x3 - x2) -
                             math.atan2(y1 - y2, x1 - x2))
        if angle < 0:
            angle += 360
            print(angle)

        # Draw
        if draw:
            cv2.line(img, (x1, y1), (x2, y2), (255, 255, 255), 3)
            cv2.line(img, (x3, y3), (x2, y2), (255, 255, 255), 3)
            cv2.circle(img, (x1, y1), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x1, y1), 15, (0, 0, 255), 2)
            cv2.circle(img, (x2, y2), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), 15, (0, 0, 255), 2)
            cv2.circle(img, (x3, y3), 10, (0, 0, 255), cv2.FILLED)
            cv2.circle(img, (x3, y3), 15, (0, 0, 255), 2)
            cv2.putText(img, str(int(angle)), (x2 - 50, y2 + 50),
                        cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 2)
        return angle

def main():
    cap = cv2.VideoCapture(0)
    pTime = 0
    detector = poseDetector()
    while True:
        success, img = cap.read()
        img = detector.findPose(img)
        lmList = detector.findPosition(img, draw=False)
        if len(lmList) != 0:
            print(lmList[14])
            cv2.circle(img, (lmList[14][1], lmList[14][2]), 15, (0, 0, 255), cv2.FILLED)

        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime

        cv2.putText(img, str(int(fps)), (70, 50), cv2.FONT_HERSHEY_PLAIN, 3,
                    (255, 0, 0), 3)

        cv2.imshow("Image", img)
        if cv2.waitKey(1) == 13: #13 is the Enter Key
            break
        
    cap.release()
    cv2.destroyAllWindows() 

if __name__ == "__main__":
    main()

[14, 180, 382]
[14, 181, 381]
[14, 181, 380]
[14, 181, 380]
[14, 181, 382]
[14, 185, 368]
[14, 187, 359]
[14, 188, 355]
[14, 187, 348]
[14, 184, 333]
[14, 180, 326]
[14, 175, 326]
[14, 175, 325]
[14, 176, 325]
[14, 176, 319]
[14, 176, 325]
[14, 179, 338]
[14, 185, 355]
[14, 188, 358]
[14, 188, 358]
[14, 188, 357]
[14, 188, 355]
[14, 183, 354]
[14, 178, 354]
[14, 175, 353]
[14, 173, 353]
[14, 172, 353]
[14, 171, 353]
[14, 183, 349]
[14, 210, 341]
[14, 209, 329]
[14, 208, 312]
[14, 205, 302]
[14, 202, 290]
[14, 200, 285]
[14, 199, 283]
[14, 199, 282]
[14, 197, 281]
[14, 197, 278]
[14, 197, 278]
[14, 196, 272]
[14, 193, 273]
[14, 190, 272]
[14, 190, 267]
[14, 187, 266]
[14, 187, 269]
[14, 198, 271]
[14, 203, 272]
[14, 205, 274]
[14, 210, 267]
[14, 210, 261]
[14, 199, 238]
[14, 198, 237]
[14, 185, 239]
[14, 185, 236]
[14, 181, 236]
[14, 177, 236]
[14, 178, 236]
[14, 179, 237]
[14, 180, 237]
[14, 181, 239]
[14, 171, 239]
[14, 168, 239]
[14, 168, 242]
[14, 166, 247]
[14, 163, 247]
[14, 163, 